In [1]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet  # Download Jan 2023 data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet  # Download Feb 2023 data

--2024-05-23 21:51:24--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.32.192.116, 13.32.192.2, 13.32.192.190, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.32.192.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-01.parquet.1’

yellow_tripdata_202 100%[===================>]  45.46M  73.3MB/s    in 0.6s    

2024-05-23 21:51:25 (73.3 MB/s) - ‘yellow_tripdata_2023-01.parquet.1’ saved [47673370/47673370]

--2024-05-23 21:51:25--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.32.192.116, 13.32.192.124, 13.32.192.2, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.32.192.116|:443... connect

In [2]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
/home/codespace/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
jan_data = pd.read_parquet("yellow_tripdata_2023-01.parquet")
jan_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


## Q1.

In [4]:
print(f"January Yellow Cab data contains {jan_data.shape[1]} columns")

January Yellow Cab data contains 19 columns


## Q2.

In [5]:
jan_data['duration'] = jan_data['tpep_dropoff_datetime'] - jan_data['tpep_pickup_datetime']
jan_data.duration = jan_data.duration.apply(lambda td: td.total_seconds() / 60)
print(f"January data's duration column has a standard deviation of {jan_data.duration.std()} mins")

January data's duration column has a standard deviation of 42.59435124195458 mins


## Q3.

In [6]:
print(f"After droping outliers, we are left with {round(jan_data[(1<=jan_data.duration) & (jan_data.duration<=60)].shape[0]/jan_data.shape[0]*100, 2)}% of original data size")
jan_data = jan_data[(1<=jan_data.duration) & (jan_data.duration<=60)]

After droping outliers, we are left with 98.12% of original data size


## Q4.

In [7]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

jan_data[categorical] = jan_data[categorical].astype(str)

train_dicts = jan_data[categorical].to_dict(orient='records')

dv = DictVectorizer()
X = dv.fit_transform(train_dicts)
print(f"Dimensionality of matrix {X.shape[1]}")
y = jan_data.duration.values

Dimensionality of matrix 515


## Q5.

In [ ]:
lr = LinearRegression()
lr.fit(X, y)

y_pred = lr.predict(X)

print(f"Train MSE: {mean_squared_error(y, y_pred, squared=False)}")

## Q6.

In [ ]:
feb_data = pd.read_parquet("yellow_tripdata_2023-02.parquet")
feb_data.head()
feb_data['duration'] = feb_data['tpep_dropoff_datetime'] - feb_data['tpep_pickup_datetime']
feb_data.duration = feb_data.duration.apply(lambda td: td.total_seconds() / 60)
feb_data = feb_data[(1<=feb_data.duration) & (feb_data.duration<=60)]

feb_data[categorical] = feb_data[categorical].astype(str)

val_dicts = feb_data[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = feb_data.duration.values
y_pred = lr.predict(X_val)

print(f"Validation MSE on February data: {mean_squared_error(y_val, y_pred, squared=False)}")